No siempre es bueno, cuanto mas caracterisiticas es mejor, no siempre funciona de la mejor manera.

# Metricas
Hay dos tipos de metricas
- Para clasificacion
- Para regresion

In [1]:
# Problema de clasificacion
import pandas as pd
import numpy as np
names_clas = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
df_clas = pd.read_csv('../Datasets/pima-indians-diabetes.csv', names=names_clas)

array_clas = df_clas.values
X_clas_clas = array_clas[:,0:8]
Y_clas_clas = array_clas[:,8]


In [2]:
# Problema de regresion
names_reg = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
df_res = pd.read_csv('../Datasets/housing.csv', names=names_reg)

array_res = df_res.values
X_clas_res = array_res[:,0:8]
Y_clas_res = array_res[:,8]

In [3]:
# # escalamos los datos
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# X_clas_scaled = scaler.fit_transform(X_clas)
# results = cross_val_score(model, X_clas_scaled, Y_clas, cv=kfold, scoring=scoring)

## Accuracy (Porcentaje de acierto)

In [4]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

kfold = KFold(n_splits=10)
model = LogisticRegression(solver='lbfgs', max_iter=1000)
scoring = 'accuracy'
results = cross_val_score(model, X_clas_clas, Y_clas_clas, cv=kfold, scoring=scoring)

print(f"Accuracy: {results.mean()} ")

Accuracy: 0.7760423786739576 


In [5]:
df_clas.groupby('class').size()

class
0    500
1    268
dtype: int64

Podemos ver que las clases estan desbalanceadas, por eso sera mejor mirar y evaluarlo con Kappa.

## Kappa

In [6]:
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split

test_size = 0.33
seed = 7
X_train, X_test, Y_train, Y_test = train_test_split(X_clas_clas, Y_clas_clas, test_size=test_size, random_state=seed)

model = LogisticRegression(solver='lbfgs', max_iter=1000)
model.fit(X_train, Y_train)
predicted = model.predict(X_test)
cohen_score = cohen_kappa_score(Y_test, predicted)
print(f"Cohens score: {cohen_score*100.0:,.2f}%")

Cohens score: 52.42%


Ahora podemos ver que el cohens score no es tan bueno y podemos ver que el modelo realmente no es muy bueno

## Aera bajo la curva de ROC
Muy importante para la clasificacion binaria.
Un area de 1.0 representa uyn modelo que hizo todas las predicciones perfectamente. Un aerea de 0.5 representa un modelo tan bueno como aleatorio.

In [9]:
# Cross validation Classification ROC AUC
kfold = KFold(n_splits=10)
model = LogisticRegression(solver='lbfgs', max_iter=1000)
scoring = 'roc_auc'
results = cross_val_score(model, X_clas_clas, Y_clas_clas, cv=kfold, scoring=scoring)
print(f"AUC: {results.mean()} {results.std()}")

AUC: 0.8280277116334988 0.0426362426387057


El AUC, es cercano a 1, es decir que no es un mal modelo.

# Matriz de confusion
Es lo mas importante al estar trabajando en un problema de classificacion.

In [10]:
from sklearn.metrics import confusion_matrix

test_size = 0.33
seed = 7
X_train, X_test, Y_train, Y_test = train_test_split(X_clas_clas, Y_clas_clas, test_size=test_size, random_state=seed)

model = LogisticRegression(solver='lbfgs', max_iter=1000)
model.fit(X_train, Y_train)
predicted = model.predict(X_test)
matriz = confusion_matrix(Y_test, predicted)
print(matriz)


[[142  20]
 [ 34  58]]


- 142 instancias me lo ha predicho correctamente para la clase 0. Sin embargo, 20 instancias me las ha predicho erroneamente como la classe 1 cuando eran de la clase 0.
- 58 me las ha predicho como la clase 1 correctamente. Sin embargo, me ha predicho  34 erronias como classe 0, cuando eran de la classe 1

Podemos ver que en la segunda clase se ha equivocado mas del 50%, ya que ha fallado 34, por tanto no es un buen modelo.

# Reporte de clasificacion